<h2>FT004 - Cuentas por Pagar </h2>
<p> En este apartado se trata la base de datos FT004 </p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [2]:
# # Se carga el FT005 completo desde el archivo .dta
# FT004_2021_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT004 20202021.dta')
# # Se carga el FT004 para diciembre de 2019 desde el archivo .dta
# FT004_2019 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT004 122019.dta')
# # Se concatenan la bases de datos 
# frames = [FT004_2021_2021, FT004_2019]
# FT004 = pd.concat(frames, ignore_index=True)
# # Se eliminan la bases que no se necesitan para liberar espacio en la memoria
# del [FT004_2021_2021, FT004_2019, frames]

In [3]:
# Se carga el FT004 completo desde el archivo .csv
FT004 = pd.read_csv('C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Bases_FT/BD_actualizadas/2_Mas_reciente_25082021/FT004.csv', dtype={'Nit':object, 'Año': object, 'Periodo': object, 'Fecha': object}, encoding='latin-1')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Se genera una nueva columna extrayendo el mes y el año de la variable fecha
FT004['ano'] = FT004['Fecha'].str[:4]
FT004['mes'] = FT004['Fecha'].str[4:6]

In [5]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004.rename(columns={'Nit': 'nit_eps', 'lineaNegocio': 'linea','tipoIdAcreedor': 'tipoidacreedor', 'idAcreedor': 'nitproveedor', 'dvAcreedor': 'DVacreedor', 'conceptoAcreencia': 'concepto', 'cxpNoVencidas': 'cxpnovencidas', 'cxpMora30dias': 'cxp30','cxpMora60dias': 'cxp60', 'cxpMora90dias': 'cxp90', 'cxpMora180dias': 'cxp180', 'cxpMora360dias': 'cxp360','cxpMoraMayor360dias': 'cxpmayor360'}, inplace=True)

In [6]:
# Se transforman los datos para su correcta identificación
FT004[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoidacreedor',
       'nitproveedor', 'DVacreedor', 'nombreAcreedor', 'actividadAcreedor',
       'concepto', 'medicionPosterior']] = FT004[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoidacreedor',
       'nitproveedor', 'DVacreedor', 'nombreAcreedor', 'actividadAcreedor',
       'concepto', 'medicionPosterior']].astype(str)

FT004[['cxpnovencidas', 'cxp30', 'cxp60',
       'cxp90', 'cxp180', 'cxp360', 'cxpmayor360', 'ajuste', 'saldo']] = FT004[['cxpnovencidas', 'cxp30', 'cxp60',
       'cxp90', 'cxp180', 'cxp360', 'cxpmayor360', 'ajuste', 'saldo']].astype(float)

In [7]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT004 = FT004.loc[FT004['linea'] == '1']

In [8]:
# Se crea la columna Cuentas por Pagar (CXP) sumando las columnas cxpnovencidas, cxp30, cxp60, cxp90, cxp180, cxp360 y cxpmayor360
CXP = FT004['cxpnovencidas'] + FT004['cxp30'] + FT004['cxp60'] + FT004['cxp90'] + FT004['cxp180'] + FT004['cxp360'] + FT004['cxpmayor360']
FT004['CXP'] = CXP

In [9]:
# Para hacer la suma de CXC por NIT EPS, tipo de proveedor, NIT proveedor, Mes y Año
FT004 = FT004.groupby(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano']).agg({'CXP': 'sum'}).reset_index()

In [10]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de deudor, NIT proveedor, Mes y Año. Para esto se cuenta el número de duplicados
duplicadosFT004 = FT004[FT004.duplicated(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT004.nit_eps.count(), sep='\n')

El número de registros duplicados es:
0


In [11]:
# Se carga la base de datos cuenta2.dta para su posterior pega
cuentas2 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cuentas2.dta')

In [12]:
# Se tranforma el formato de las columnas del dataframe cuentas2
cuentas2[['v2', 'nitproveedor']] = cuentas2[['v2', 'nitproveedor']].astype(str)

In [13]:
# Se comprueba que no haya duplicados cuando se filtra por Nitproveedor. Para esto se cuenta el número de duplicados
duplicados_cuentas2 = cuentas2[cuentas2.duplicated(['nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicados_cuentas2.nitproveedor.count(), sep='\n')

El número de registros duplicados es:
0


In [14]:
# Se unen las dos bases de datos para obtener la información de FT004 con cuenta2
FT004= pd.merge(FT004, cuentas2, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

<p> Esto corresponde a la cuenta del pasivo cuando se agregan las cuentas por pagar. Por tal motivo, se elimina ya que no se pueden identificar los proveedores. </p>

In [15]:
# Se encuentran las observaciones que cumplen las condiciones anteriores
df_filtered = FT004[(FT004['tipoidacreedor'] == 'OT') & (FT004['v2'] == 'cuenta')].index

In [16]:
# Se eliminan las observaciones que cumplen las condiciones anteriores
FT004 = FT004.drop(df_filtered)

In [17]:
# Para hacer la suma de CXP por NIT EPS, tipo de acreedor corregido, NIT proveedor, Mes y Año
FT004 = FT004.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).agg({'CXP': 'sum'}).reset_index()

In [18]:
FT004.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5898306 entries, 0 to 5898305
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   nit_eps       object 
 1   nitproveedor  object 
 2   mes           object 
 3   ano           object 
 4   CXP           float64
dtypes: float64(1), object(4)
memory usage: 225.0+ MB


<p> Desde aquí se guarda la base de datos con fechas máximas por proveedor </p>

In [19]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT004_FECHA = FT004[
    (FT004['ano'] == '2021')
    ]

In [20]:
# Se transforman las columnas de mes y año para su correcta identificación
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(int)

In [21]:
del [CXP, cuentas2, df_filtered, duplicados_cuentas2, duplicadosFT004]

In [22]:
# Se procede a eliminar los duplicados dejando el último movimiento registrado. Esto se debe ya que se busca trabajar con el último dato disponible
FT004_FECHA = FT004_FECHA.sort_values(['ano', 'mes']).drop_duplicates('nitproveedor', keep='last')

In [23]:
# Para guardar la base de datos lista para ser tratada en formato CSV
FT004_FECHA.to_excel(r'C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Jul 6 Pagos WEB/Pagos WEB/EPS/2021/UNION_BASES/FT004_FECHA.xlsx', index = False)

<p> Hasta aquí se guarda la base de datos con fechas máximas por proveedor </p>

In [24]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT004 = FT004.set_index('mes').groupby(['nit_eps', 'nitproveedor', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'nitproveedor', 'ano'], 1).reset_index()

In [25]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT004[
    (FT004['ano'] == 2019) & (FT004['mes'] == 1) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 2) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 3) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 4) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 5) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 6) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 7) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 8) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 9) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 10) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 11)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004 = FT004.drop(df_filtered)

In [26]:
# Se completan los años y los meses para las observaciones que no presentaron ninguna observación en un año de interés. Por ejemplo, a aquellos sujetos que reportaron cifras en 2019, pero no en 2020, se les completa el año 2020 para poder calcular la diferencia entre diciembre de 2019 y enero de 2020
# Se convierte la variable en mes y año
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(float)
FT004['periodo'] = pd.to_datetime(FT004.ano*10000 + FT004.mes*100 + 1, format='%Y%m%d')
fecha_ideal = pd.date_range('12-01-2019','07-01-2021',freq='M')

<p> Desde aquí se filtra la base por la fecha máxima de reporte de la información </p>

In [27]:
# Se seleccionan las columnas requeridas antes del cruce para filtrar la información (se excluye el valor de CXC)
FT004_FECHA = FT004_FECHA[['nit_eps', 'nitproveedor', 'mes', 'ano']]

In [28]:
# Se modifican los nombres de las columnas para posterimente pegar las bases de datos
FT004_FECHA.rename(columns={'mes': 'ULTIMO_MES_FT004', 'ano': 'ULTIMO_ANO_FT004'}, inplace=True)

In [31]:
FT004.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12321524 entries, 0 to 12321523
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nit_eps           object        
 1   nitproveedor      object        
 2   ano               float64       
 3   mes               float64       
 4   CXP               float64       
 5   periodo           datetime64[ns]
 6   ULTIMO_MES_FT004  object        
 7   ULTIMO_ANO_FT004  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 846.1+ MB


In [34]:
# Se crea una variable de control para filtrar por fecha del reporte
FT004_FECHA[['ULTIMO_ANO_FT004', 'ULTIMO_MES_FT004']] = FT004_FECHA[['ULTIMO_ANO_FT004', 'ULTIMO_MES_FT004']].astype(float)
FT004_FECHA['periodo_filtro'] = pd.to_datetime(FT004_FECHA.ULTIMO_ANO_FT004*10000 + FT004_FECHA.ULTIMO_MES_FT004*100 + 1, format='%Y%m%d')

In [29]:
# Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y prestadores
FT004 = pd.merge(FT004, FT004_FECHA, how='left', left_on=['nit_eps', 'nitproveedor'], right_on=['nit_eps', 'nitproveedor'])

In [39]:
FT004.columns

Index(['nit_eps', 'nitproveedor', 'ano', 'mes', 'CXP', 'periodo',
       'ULTIMO_MES_FT004', 'ULTIMO_ANO_FT004', 'periodo_filtro'],
      dtype='object')

In [36]:
# Se seleccionan las columnas requeridas antes del cruce para filtrar la información (se excluye el valor de CXC)
FECHA = FT004_FECHA[['nit_eps', 'nitproveedor', 'periodo_filtro']]

In [38]:
# Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y prestadores
FT004 = pd.merge(FT004, FECHA, how='left', left_on=['nit_eps', 'nitproveedor'], right_on=['nit_eps', 'nitproveedor'])

In [41]:
del [fecha_ideal, FECHA, df_filtered, FT004_FECHA]

In [40]:
# Para guardar la base de datos lista para ser tratada en formato CSV
FT004.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT004_sin_tratar.csv', index = False, encoding='utf-8-sig')

<p>Cjeck point</p>

In [43]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones. La idea es eliminar aquellas observaciones que están por fuera de las fechas máximas de reporte de cada EPS, es decir, si una EPS reportó su información 
FT004 = FT004[
    (FT004['periodo'] <= FT004['periodo_filtro'])
    ]

In [48]:
# Se agrupa la información según NIT EPS, NIT proveedor y periodo sumando el valor de CXP que corresponda a cada agrupación descrita
FT004_dif = FT004.groupby(['nit_eps','nitproveedor', pd.Grouper(key='periodo', freq='M')])[['CXP']].sum().reset_index()

In [49]:
fecha_ideal = pd.date_range('12-01-2019','07-01-2021',freq='M')

In [50]:
# Se prepara el dataframe para calcular la diferencia periodo a periodo (t+1 - t) respetando la agregación anterior
FT004_dif = FT004_dif.set_index('periodo').groupby(['nit_eps', 'nitproveedor']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'nitproveedor'], 1).reset_index()

In [ ]:
FT004_dif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13747089 entries, 0 to 13747088
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   nit_eps       object        
 1   nitproveedor  object        
 2   level_2       datetime64[ns]
 3   CXP           float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 419.5+ MB


In [51]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_dif.rename(columns={'level_2': 'periodo'}, inplace=True)

In [52]:
# Se establece el index (identificación) como NIT EPS, NIT proveedor y el periodo para cada observación
FT004_dif = FT004_dif.set_index(['nit_eps', 'nitproveedor', 'periodo'])

In [53]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT004_dif = FT004_dif.diff().fillna(0).reset_index()

In [54]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_dif.rename(columns={'CXP': 'DCXPmes'}, inplace=True)

#  Se extrae el mes y el año para porterior left join con el agregado
FT004_dif['ano'] = FT004_dif['periodo'].dt.year
FT004_dif['mes'] = FT004_dif['periodo'].dt.month

#Se transforman las variables a STR para mejorar su manipulación
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(int)

# Se eliminan las columnas que no son de interés
FT004 = FT004.drop(['periodo'], axis=1)
FT004_dif = FT004_dif.drop(['periodo'], axis=1)

In [55]:
# Se unen las dos bases de datos para obtener la información de webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y FT005 en una sola base de datos. Desagregando el valor de VCausado y el VPagado para cada mes
FT004 = pd.merge(FT004_dif, FT004, how='left', left_on=['nit_eps', 'nitproveedor', 'mes', 'ano'], right_on=['nit_eps', 'nitproveedor', 'mes', 'ano'])

In [56]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT004[
    (FT004['ano'] == 2019)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004 = FT004.drop(df_filtered)

In [57]:
# Se libera espacio de la memoria
del [df_filtered, fecha_ideal, FT004_dif]

In [59]:
# Se convierte la variable en mes y año
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(float).astype(int)

In [ ]:
# EL DF ES MI BACK UP

In [60]:
# Se agrega la información por NIT_EPS, Nitproveedor, noID, Departamento, Ano, Mes para las variables VPagado, VCausado, PW, Giro, No_UPC, Compra_Cartera
FT004 = FT004.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).agg({'CXP':'sum', 'DCXPmes':'sum'}).reset_index()

<p> Desde aquí se filtra la base por la fecha máxima de reporte de la información </p>

In [61]:
# Se seleccionan las columnas requeridas antes del cruce para filtrar la información (se excluye el valor de CXC)
FT004_FECHA = FT004_FECHA[['nit_eps', 'nitproveedor', 'mes', 'ano']]

NameError: name 'FT004_FECHA' is not defined

In [ ]:
FT004_FECHA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323360 entries, 27 to 5898258
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   nit_eps       323360 non-null  object 
 1   nitproveedor  323360 non-null  object 
 2   mes           323360 non-null  object 
 3   ano           323360 non-null  object 
 4   CXP           323360 non-null  float64
dtypes: float64(1), object(4)
memory usage: 14.8+ MB


In [ ]:
# Se modifican los nombres de las columnas para posterimente pegar las bases de datos
FT004_FECHA.rename(columns={'mes': 'ULTIMO_MES_FT004', 'ano': 'ULTIMO_ANO_FT004'}, inplace=True)

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y prestadores
FT004 = pd.merge(FT004, FT004_FECHA, how='left', left_on=['nit_eps', 'nitproveedor'], right_on=['nit_eps', 'nitproveedor'])

In [62]:
FT004.columns

Index(['nit_eps', 'nitproveedor', 'DCXPmes', 'ano', 'mes', 'CXP',
       'ULTIMO_MES_FT004', 'ULTIMO_ANO_FT004', 'periodo_filtro'],
      dtype='object')

In [63]:
# Se crea una variable de control para filtrar por fecha del reporte
FT004['periodo'] = pd.to_datetime(FT004.ano*10000 + FT004.mes*100 + 1, format='%Y%m%d')
# FT004['periodo_filtro'] = pd.to_datetime(FT004.ULTIMO_ANO_FT004*10000 + FT004.ULTIMO_MES_FT004*100 + 1, format='%Y%m%d')

In [65]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones. La idea es eliminar aquellas observaciones que están por fuera de las fechas máximas de reporte de cada EPS, es decir, si una EPS reportó su información 
FT004 = FT004[
    (FT004['periodo'] <= FT004['periodo_filtro'])
    ]

In [66]:
del [FT004_dif, df]

In [67]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
# FT004_V2.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/carteraprestador_2020_2021.xlsx', index = False)
FT004.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/carteraprestador_2020_ene_jul_2021.csv', index = False, encoding='utf-8-sig')

In [40]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT004]